In [60]:
%run 'D:\dev\prod\metrikaHelper.py'
%run 'D:\dev\prod\allStuffHelper.py'
import pandas as pd
import psycopg2
import math
import numpy
from datetime import timedelta
from datetime import datetime
from pandas import ExcelWriter
import json,requests,time
from bs4 import BeautifulSoup
from pandas import ExcelWriter
import numpy as np

In [ ]:
def getMetrikaData (projectInfo,date1,date2):
    dimensions = 'ym:s:directID,ym:s:deviceCategory,ym:s:gender,ym:s:ageInterval'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(projectInfo['projectid'])])
    otherMetrics = 'ym:s:visits,ym:s:bounceRate'
    metrics = ','.join([otherMetrics,goalsMetrics])
    filters = ""
    attribution = 'lastsign'
    metrikaData = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,projectInfo['token'],projectInfo['counterId'],filters,attribution)
    metrikaGoalsData = mergeCPAVisitsColumns(dropTotalRow(metrikaData))
    return metrikaGoalsData

In [4]:
project = u'stranao3.ru'
VipProjectInfo = getExtendedProjectInfo(project,'Metrika')
print VipProjectInfo

{'domain': 'stranao3.ru', 'metrikapassword': 'pass-RookeeXML', 'metrikalogin': 'RookeeXML', 'projectid': 2450701L, 'counterId': 1033133, 'token': u'AQAAAAANKA8jAAFxmIk5swbJYEETp-gfQArNcpI'}


In [9]:
date1 = '2016-02-01'
date2 = '2016-07-25'
df = getMetrikaData(VipProjectInfo,date1,date2)

In [15]:
df

,"﻿""Кампания Яндекс.Директа""",device_type,gender,age,visits,bounceRate,goal_visits
0,N-18376036,ПК,женский,35‑44 года,317,0.110410,0.0
1,N-18376036,ПК,женский,25‑34 года,268,0.097015,0.0
2,N-18376036,ПК,женский,45 лет и старше,253,0.154150,0.0
3,N-18249634,ПК,женский,35‑44 года,169,0.414201,0.0
4,N-18249634,ПК,женский,25‑34 года,129,0.317829,0.0
5,N-18376036,ПК,мужской,25‑34 года,128,0.195312,0.0
6,N-18376036,ПК,мужской,45 лет и старше,122,0.196721,0.0
7,N-18784613,ПК,женский,45 лет и старше,121,0.214876,0.0
8,N-18249634,ПК,женский,45 лет и старше,109,0.348624,0.0
9,N-18784613,ПК,женский,25‑34 года,105,0.209524,0.0


In [16]:
df.rename(columns={u'﻿"Кампания Яндекс.Директа"': 'campaignId',
                   u'Тип устройства':'device_type',
                   u'Пол': 'gender',
                   u'Возраст': 'age',
                  u'Визиты':'visits',
                  u'Отказы':'bounceRate'},
          inplace=True)
df = df[~df[u'device_type'].isin([u'ТВ'])]
df['bounces'] = df['visits']*df['bounceRate']
df['conversion_rate'] = df['goal_visits']/df['visits']
df['campaignId'] = df['campaignId'].str.replace('N-','').astype(int)
df.replace(to_replace={'device_type':{u'ПК':'desktop',u'Смартфоны':'mobile',u'Планшеты':'tablet'},
                      'age':{u'младше 18 лет':'AGE_0_17',u'18‑24 года':'AGE_18_24',u'25‑34 года':'AGE_25_34',u'35‑44 года':'AGE_35_44',u'45 лет и старше':'AGE_45',u'Не определено':'AGE_UNKNOWN'},
                      'gender':{u'женский':'GENDER_FEMALE',u'мужской':'GENDER_MALE',u'Не определено':'GENDER_UNKNOWN'}},inplace=True)

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [20]:
DirectToken = GetToken('alexgrtula','Direct')

In [22]:
URL = 'https://api.direct.yandex.ru/live/v4/json/'
TOKEN = DirectToken

def createNewReportPayload(campaignID,startDate,endDate):
   payload = {
      "method": "CreateNewReport",
      "token": TOKEN,
      "param": {
         "CampaignID": campaignID,
         "StartDate": startDate,
         "EndDate": endDate,
         "GroupByColumns": ['clDemographics','clDeviceType'],
         "GroupByDate": "day",
         "TypeResultReport": "xml",
         "CompressReport": 0,
      }
   }
   return payload

def createJsonUTF8Payload(payload):
   return json.dumps(payload, ensure_ascii=False).encode('utf8')

def createReport(campaignID,startDate,endDate):
   payload = createJsonUTF8Payload(createNewReportPayload(campaignID,startDate,endDate))
   return requests.post(URL,payload).text

def getReportList():
   payload = createJsonUTF8Payload({
   "method": "GetReportList",
   "token": TOKEN
   })
   return requests.post(URL,payload).text

def getReport(reportID):
   reportList = getReportList()
   reportItem = (item for item in json.loads(reportList)['data'] if item["ReportID"] == json.loads(reportID)['data']).next()
   if reportItem['StatusReport'] == 'Done':
      data = requests.get(reportItem['Url']).text
      return data
   else:
      return False

def deleteReport(reportID):
   payload = createJsonUTF8Payload({
   "method": "DeleteReport",
   "token": TOKEN,
   "param": reportID
})
   return requests.post(URL,payload).text

In [46]:
direct_stats = pd.DataFrame()
for campaignId in [x for x in list(df['campaignId'].unique()) if x != 9208736]:
    print 'Create report:', campaignId
    reportId = createReport(int(campaignId),date1,date2)
    time.sleep(100)
    report = getReport(reportId)
    print deleteReport(json.loads(reportId)['data'])
    soup = BeautifulSoup(report,'lxml')
    length = int(soup.report.stat.attrs['rows'])
    print 'Direct report length = {0}\t'.format(length)
    if length > 0:
        stats = []
        for tag in soup.report.stat.find_all('row'):
            stats.append(tag.attrs)
        print 'Length check: {0}'.format(str(len(stats)==int(soup.report.stat.attrs['rows'])))
        campaignId = soup.report.campaignid.get_text()
        campaign_stats = pd.DataFrame(stats)
        campaign_stats['campaignId'] = campaignId
#         campaign_stats.replace(to_replace={'device_type':{'tablet':'mobile'}},inplace=True)
        direct_stats = pd.concat([direct_stats,campaign_stats])
    print 'Total Length =',len(direct_stats)

Create report: 18376036
{"data":1}
Direct report length = 54	
Length check: True
Total Length = 54
Create report: 18249634
{"data":1}
Direct report length = 52	
Length check: True
Total Length = 106
Create report: 18784613
{"data":1}
Direct report length = 54	
Length check: True
Total Length = 160
Create report: 18249606
{"data":1}
Direct report length = 53	
Length check: True
Total Length = 213
Create report: 18784598
{"data":1}
Direct report length = 46	
Length check: True
Total Length = 259
Create report: 18249734
{"data":1}
Direct report length = 49	
Length check: True
Total Length = 308
Create report: 18784595
{"data":1}
Direct report length = 51	
Length check: True
Total Length = 359
Create report: 18249690
{"data":1}
Direct report length = 49	
Length check: True
Total Length = 408
Create report: 19395044
{"data":1}
Direct report length = 35	
Length check: True
Total Length = 443
Create report: 18784609
{"data":1}
Direct report length = 42	
Length check: True
Total Length = 485
C

In [45]:
[x for x in list(df['campaignId'].unique()) if x != 9208736]

[18376036,
 18249634,
 18784613,
 18249606,
 18784598,
 18249734,
 18784595,
 18249690,
 19395044,
 18784609,
 19928995,
 18376043]

In [47]:
direct_stats['clicks'] = direct_stats['clicks'].astype(int)
direct_stats['clicks_context'] = direct_stats['clicks_context'].astype(int)
direct_stats['clicks_search'] = direct_stats['clicks_search'].astype(int)
direct_stats['shows'] = direct_stats['shows'].astype(int)
direct_stats['shows_context'] = direct_stats['shows_context'].astype(int)
direct_stats['shows_search'] = direct_stats['shows_search'].astype(int)
direct_stats['sum'] = direct_stats['sum'].astype(float)*30.0
direct_stats['sum_context'] = direct_stats['sum_context'].astype(float)*30.0
direct_stats['sum_search'] = direct_stats['sum_search'].astype(float)*30.0
direct_stats['campaignId'] = direct_stats['campaignId'].astype(int)

In [49]:
merged_df = pd.merge(df,direct_stats,on=['age','device_type','gender','campaignId'],how='inner')
merged_df['cpv'] = merged_df['sum']/merged_df['visits']
print 'Check merged dataframe length:',len(df)==len(merged_df)

Check merged dataframe length: False


In [50]:
merged_df

,campaignId,device_type,gender,age,visits,bounceRate,goal_visits,bounces,conversion_rate,clicks,clicks_context,clicks_search,shows,shows_context,shows_search,sum,sum_context,sum_search,cpv
0,18376036,desktop,GENDER_FEMALE,AGE_35_44,317,0.110410,0.0,35.0,0.000000,389,389,0,388116,388116,0,14260.5,14260.5,0.0,44.985804
1,18376036,desktop,GENDER_FEMALE,AGE_25_34,268,0.097015,0.0,26.0,0.000000,143,143,0,333110,333110,0,4937.1,4937.1,0.0,18.422015
2,18376036,desktop,GENDER_FEMALE,AGE_45,253,0.154150,0.0,39.0,0.000000,313,313,0,361546,361546,0,11022.9,11022.9,0.0,43.568775
3,18249634,desktop,GENDER_FEMALE,AGE_35_44,169,0.414201,0.0,70.0,0.000000,179,0,179,3723,0,3723,6987.3,0.0,6987.3,41.344970
4,18249634,desktop,GENDER_FEMALE,AGE_25_34,129,0.317829,0.0,41.0,0.000000,63,0,63,1515,0,1515,2281.5,0.0,2281.5,17.686047
5,18376036,desktop,GENDER_MALE,AGE_25_34,128,0.195312,0.0,25.0,0.000000,25,25,0,87861,87861,0,797.7,797.7,0.0,6.232031
6,18376036,desktop,GENDER_MALE,AGE_45,122,0.196721,0.0,24.0,0.000000,34,34,0,67219,67219,0,1094.4,1094.4,0.0,8.970492
7,18784613,desktop,GENDER_FEMALE,AGE_45,121,0.214876,0.0,26.0,0.000000,168,168,0,273345,273345,0,5418.3,5418.3,0.0,44.779339
8,18249634,desktop,GENDER_FEMALE,AGE_45,109,0.348624,0.0,38.0,0.000000,112,0,112,2736,0,2736,4170.6,0.0,4170.6,38.262385
9,18784613,desktop,GENDER_FEMALE,AGE_25_34,105,0.209524,0.0,22.0,0.000000,42,42,0,168767,168767,0,1154.7,1154.7,0.0,10.997143


In [59]:
MAX_BID_ADJ = 119
MIN_BID_ADJ = 6
for min_bid_adjsutment in xrange(MIN_BID_ADJ,10):
    for max_bid_adjsutment in xrange(11,MAX_BID_ADJ+1):
        print min_bid_adjsutment/10.0,max_bid_adjsutment/10.0

0.6 1.1
0.6 1.2
0.6 1.3
0.6 1.4
0.6 1.5
0.6 1.6
0.6 1.7
0.6 1.8
0.6 1.9
0.6 2.0
0.6 2.1
0.6 2.2
0.6 2.3
0.6 2.4
0.6 2.5
0.6 2.6
0.6 2.7
0.6 2.8
0.6 2.9
0.6 3.0
0.6 3.1
0.6 3.2
0.6 3.3
0.6 3.4
0.6 3.5
0.6 3.6
0.6 3.7
0.6 3.8
0.6 3.9
0.6 4.0
0.6 4.1
0.6 4.2
0.6 4.3
0.6 4.4
0.6 4.5
0.6 4.6
0.6 4.7
0.6 4.8
0.6 4.9
0.6 5.0
0.6 5.1
0.6 5.2
0.6 5.3
0.6 5.4
0.6 5.5
0.6 5.6
0.6 5.7
0.6 5.8
0.6 5.9
0.6 6.0
0.6 6.1
0.6 6.2
0.6 6.3
0.6 6.4
0.6 6.5
0.6 6.6
0.6 6.7
0.6 6.8
0.6 6.9
0.6 7.0
0.6 7.1
0.6 7.2
0.6 7.3
0.6 7.4
0.6 7.5
0.6 7.6
0.6 7.7
0.6 7.8
0.6 7.9
0.6 8.0
0.6 8.1
0.6 8.2
0.6 8.3
0.6 8.4
0.6 8.5
0.6 8.6
0.6 8.7
0.6 8.8
0.6 8.9
0.6 9.0
0.6 9.1
0.6 9.2
0.6 9.3
0.6 9.4
0.6 9.5
0.6 9.6
0.6 9.7
0.6 9.8
0.6 9.9
0.6 10.0
0.6 10.1
0.6 10.2
0.6 10.3
0.6 10.4
0.6 10.5
0.6 10.6
0.6 10.7
0.6 10.8
0.6 10.9
0.6 11.0
0.6 11.1
0.6 11.2
0.6 11.3
0.6 11.4
0.6 11.5
0.6 11.6
0.6 11.7
0.6 11.8
0.6 11.9
0.7 1.1
0.7 1.2
0.7 1.3
0.7 1.4
0.7 1.5
0.7 1.6
0.7 1.7
0.7 1.8
0.7 1.9
0.7 2.0
0.7 2.1
0.7 2.2
0.7 2.3
0.7 

In [116]:
def calcGenderAgeDfForCampaign(campaignDf):
    calc_gender_age_df = campaignDf[~campaignDf['gender'].isin(['GENDER_UNKNOWN'])]
    calc_gender_age_df = calc_gender_age_df[~calc_gender_age_df['age'].isin(['AGE_UNKNOWN'])]
    calc_gender_age_df = calc_gender_age_df[['campaignId','age','gender','sum','visits','goal_visits']].groupby(['campaignId','age','gender'],as_index=False).sum()
    calc_gender_age_df['conversed_cpa'] = calc_gender_age_df['goal_visits']/calc_gender_age_df['sum']
    calc_gender_age_df['conversed_cpa'] = calc_gender_age_df['conversed_cpa'].replace(np.inf, np.nan).fillna(0)
    calc_gender_age_df['conversion_rate'] = calc_gender_age_df['goal_visits']/calc_gender_age_df['visits']
    calc_gender_age_df['cpv'] = calc_gender_age_df['sum']/calc_gender_age_df['visits']
    calc_gender_age_visits = calc_gender_age_df['visits'].sum()
    calc_gender_age_sum = calc_gender_age_df['sum'].sum()
    calc_gender_age_goal_visits = calc_gender_age_df['goal_visits'].sum()
    calc_gender_age_conversion_rate = calc_gender_age_goal_visits/calc_gender_age_visits
    try:
        calc_gender_age_conversed_cpa = calc_gender_age_goal_visits/calc_gender_age_sum
    except ZeroDivisionError:
        calc_gender_age_conversed_cpa = 0
    calc_gender_age_conversed_cpa_std = calc_gender_age_df['conversed_cpa'].std()
    try:
        calc_gender_age_cpa = calc_gender_age_sum/calc_gender_age_goal_visits
    except ZeroDivisionError:
        calc_gender_age_cpa = 0
    calc_gender_age_df['sqrt_sum'] = calc_gender_age_df['sum'].apply(numpy.sqrt)
    calc_gender_age_df['z_criteria'] = (calc_gender_age_df['conversed_cpa'] - calc_gender_age_conversed_cpa)/(calc_gender_age_conversed_cpa_std/calc_gender_age_df['sqrt_sum'])
    calc_gender_age_df['z_criteria'] = calc_gender_age_df['z_criteria'].replace(np.inf, np.nan).fillna(0)
    return calc_gender_age_df

def calcGenderAgeDf(merged_df):
    campaign_dfs = []
    for campaignId in merged_df['campaignId'].unique():
        calc_gender_age_df = calcGenderAgeDfForCampaign(merged_df[merged_df['campaignId'] == campaignId])
        campaign_dfs.append(calc_gender_age_df)
    return campaign_dfs

def calcBidAdjustments(campaign_dfs,ga_min_bid_adjustment,ga_max_bid_adjustment):
    calculated_dfs = []
    for calc_gender_age_df in campaign_dfs:
        calc_gender_age_df['min_bid'] = ga_min_bid_adjustment
        calc_gender_age_df['max_bid'] = ga_max_bid_adjustment
        ga_maximum_z_criteria = calc_gender_age_df['z_criteria'].max()
        ga_minimum_z_criteria = calc_gender_age_df['z_criteria'].min()
        ga_delta_z_criteria = ga_maximum_z_criteria - ga_minimum_z_criteria
        ga_adjustment_step = (ga_max_bid_adjustment-ga_min_bid_adjustment)/ga_delta_z_criteria
        calc_gender_age_df['bid_adjustment'] = (calc_gender_age_df['z_criteria'] + abs(ga_minimum_z_criteria))*ga_adjustment_step + ga_min_bid_adjustment
        for i in xrange(0,len(calc_gender_age_df)):
            if (calc_gender_age_df.loc[i,'age'] == 'AGE_UNKNOWN') or (calc_gender_age_df.loc[i,'gender'] == 'GENDER_UNKNOWN'):
                calc_gender_age_df.loc[i,'bid_adjustment'] = 1
        calc_gender_age_df['new_visits'] = calc_gender_age_df['visits']*calc_gender_age_df['bid_adjustment']
        calc_gender_age_df['new_cpv'] = calc_gender_age_df['cpv']*calc_gender_age_df['bid_adjustment']
        calc_gender_age_df['new_sum'] = calc_gender_age_df['new_visits']*calc_gender_age_df['new_cpv']
        calc_gender_age_df['new_goal_visits'] = calc_gender_age_df['new_visits']*calc_gender_age_df['conversion_rate']
        calc_gender_age_df['new_conversed_cpa'] = calc_gender_age_df['new_goal_visits']/calc_gender_age_df['new_sum']
        calc_gender_age_df['new_conversed_cpa'] = calc_gender_age_df['new_conversed_cpa'].replace(np.inf, np.nan).fillna(0)
        calc_gender_age_new_visits = calc_gender_age_df['new_visits'].sum()
        calc_gender_age_new_sum = calc_gender_age_df['new_sum'].sum()
        calc_gender_age_new_goal_visits = calc_gender_age_df['new_goal_visits'].sum()
        try:
            calc_gender_age_new_conversion_rate = calc_gender_age_new_goal_visits/calc_gender_age_new_visits
        except ZeroDivisionError:
            calc_gender_age_new_conversion_rate = 0
        try:
            calc_gender_age_new_conversed_cpa = calc_gender_age_new_goal_visits/calc_gender_age_new_sum
        except ZeroDivisionError:
            calc_gender_age_new_conversed_cpa = 0
        calc_gender_age_new_conversed_cpa_std = calc_gender_age_df['new_conversed_cpa'].std()
        try:
            calc_gender_age_new_cpa = calc_gender_age_new_sum/calc_gender_age_new_goal_visits
        except ZeroDivisionError:
            calc_gender_age_new_cpa = 0 
        calculated_dfs.append(calc_gender_age_df)
    return calculated_dfs

In [120]:
calcGenderAgeDf(merged_df)[2]

,campaignId,age,gender,sum,visits,goal_visits,conversed_cpa,conversion_rate,cpv,sqrt_sum,z_criteria
0,18784613,AGE_0_17,GENDER_FEMALE,898.8,73,0.0,0.000000,0.000000,12.312329,29.979993,-10.389280
1,18784613,AGE_0_17,GENDER_MALE,246.9,33,0.0,0.000000,0.000000,7.481818,15.713052,-5.445208
2,18784613,AGE_18_24,GENDER_FEMALE,1899.3,78,0.0,0.000000,0.000000,24.350000,43.580959,-15.102565
3,18784613,AGE_18_24,GENDER_MALE,268.2,60,0.0,0.000000,0.000000,4.470000,16.376813,-5.675228
4,18784613,AGE_25_34,GENDER_FEMALE,2760.3,170,0.0,0.000000,0.000000,16.237059,52.538557,-18.206735
5,18784613,AGE_25_34,GENDER_MALE,4123.8,115,1.0,0.000242,0.008696,35.859130,64.216820,36.990788
6,18784613,AGE_35_44,GENDER_FEMALE,1490.1,131,0.0,0.000000,0.000000,11.374809,38.601813,-13.377090
7,18784613,AGE_35_44,GENDER_MALE,1215.9,49,1.0,0.000822,0.020408,24.814286,34.869758,97.022061
8,18784613,AGE_45,GENDER_FEMALE,6725.1,153,0.0,0.000000,0.000000,43.954902,82.006707,-28.418641
9,18784613,AGE_45,GENDER_MALE,2328.6,116,0.0,0.000000,0.000000,20.074138,48.255570,-16.722507


In [62]:
dfs1 = calcGenderAgeDf(merged_df)

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [103]:
dfs1[1]

,campaignId,age,gender,sum,visits,goal_visits,conversed_cpa,conversion_rate,cpv,sqrt_sum,z_criteria,min_bid,max_bid,bid_adjustment,new_visits,new_cpv,new_sum,new_goal_visits,new_conversed_cpa
0,18249634,AGE_0_17,GENDER_FEMALE,6207.9,360,0.0,0.000000,0.000000,17.244167,78.790228,-16.514479,0.9,11.9,1.534182,552.305391,26.455684,14611.616842,0.000000,0.000000
1,18249634,AGE_0_17,GENDER_MALE,1238.7,114,0.0,0.000000,0.000000,10.865789,35.195170,-7.376929,0.9,11.9,2.427981,276.789876,26.381934,7302.252349,0.000000,0.000000
2,18249634,AGE_18_24,GENDER_FEMALE,3547.8,305,0.0,0.000000,0.000000,11.632131,59.563412,-12.484527,0.9,11.9,1.928376,588.154661,22.431122,13192.968842,0.000000,0.000000
3,18249634,AGE_18_24,GENDER_MALE,871.8,219,1.0,0.001147,0.004566,3.980822,29.526259,89.458015,0.9,11.9,11.900000,2606.100000,47.371781,123455.598000,11.900000,0.000096
4,18249634,AGE_25_34,GENDER_FEMALE,17484.6,1039,3.0,0.000172,0.002887,16.828296,132.229346,36.357197,0.9,11.9,6.705884,6967.413451,112.848603,786262.877558,20.117652,0.000026
5,18249634,AGE_25_34,GENDER_MALE,7232.7,533,1.0,0.000138,0.001876,13.569794,85.045282,15.381360,0.9,11.9,4.654109,2480.640054,63.155298,156665.560608,4.654109,0.000030
6,18249634,AGE_35_44,GENDER_FEMALE,33580.2,1046,1.0,0.000030,0.000956,32.103442,183.249011,-22.997885,0.9,11.9,0.900000,941.400000,28.893098,27199.962000,0.900000,0.000033
7,18249634,AGE_35_44,GENDER_MALE,3185.7,274,1.0,0.000314,0.003650,11.626642,56.442006,38.204981,0.9,11.9,6.886627,1886.935789,80.068349,151083.832637,6.886627,0.000046
8,18249634,AGE_45,GENDER_FEMALE,29888.7,750,1.0,0.000033,0.001333,39.851600,172.883487,-19.901259,0.9,11.9,1.202900,902.174951,47.937487,43248.000051,1.202900,0.000028
9,18249634,AGE_45,GENDER_MALE,4551.3,448,0.0,0.000000,0.000000,10.159152,67.463323,-14.140353,0.9,11.9,1.766409,791.351427,17.945222,14200.976703,0.000000,0.000000


In [82]:
dfs2 = calcBidAdjustments(dfs1,0.8,1.3)

In [83]:
dfs2[11]

,campaignId,age,gender,sum,visits,goal_visits,conversed_cpa,conversion_rate,cpv,sqrt_sum,z_criteria,min_bid,max_bid,bid_adjustment,new_visits,new_cpv,new_sum,new_goal_visits,new_conversed_cpa
0,18376043,AGE_0_17,GENDER_FEMALE,6207.9,360,0.0,0.000000,0.000000,17.244167,78.790228,-16.514479,0.8,1.3,0.828826,298.377518,14.292421,4264.537172,0.000000,0.000000
1,18376043,AGE_0_17,GENDER_MALE,1238.7,114,0.0,0.000000,0.000000,10.865789,35.195170,-7.376929,0.8,1.3,0.869454,99.117722,9.447301,936.394935,0.000000,0.000000
2,18376043,AGE_18_24,GENDER_FEMALE,3547.8,305,0.0,0.000000,0.000000,11.632131,59.563412,-12.484527,0.8,1.3,0.846744,258.257030,9.849441,2543.687497,0.000000,0.000000
3,18376043,AGE_18_24,GENDER_MALE,871.8,219,1.0,0.001147,0.004566,3.980822,29.526259,89.458015,0.8,1.3,1.300000,284.700000,5.175068,1473.342000,1.300000,0.000882
4,18376043,AGE_25_34,GENDER_FEMALE,17484.6,1039,3.0,0.000172,0.002887,16.828296,132.229346,36.357197,0.8,1.3,1.063904,1105.396066,17.903689,19790.667184,3.191711,0.000161
5,18376043,AGE_25_34,GENDER_MALE,7232.7,533,1.0,0.000138,0.001876,13.569794,85.045282,15.381360,0.8,1.3,0.970641,517.351821,13.171402,6814.248969,0.970641,0.000142
6,18376043,AGE_35_44,GENDER_FEMALE,33580.2,1046,1.0,0.000030,0.000956,32.103442,183.249011,-22.997885,0.8,1.3,0.800000,836.800000,25.682753,21491.328000,0.800000,0.000037
7,18376043,AGE_35_44,GENDER_MALE,3185.7,274,1.0,0.000314,0.003650,11.626642,56.442006,38.204981,0.8,1.3,1.072119,293.760718,12.465149,3661.771085,1.072119,0.000293
8,18376043,AGE_45,GENDER_FEMALE,29888.7,750,1.0,0.000033,0.001333,39.851600,172.883487,-19.901259,0.8,1.3,0.813768,610.326134,32.429964,19792.854533,0.813768,0.000041
9,18376043,AGE_45,GENDER_MALE,4551.3,448,0.0,0.000000,0.000000,10.159152,67.463323,-14.140353,0.8,1.3,0.839382,376.043247,8.527412,3206.675564,0.000000,0.000000


In [121]:
MAX_BID_ADJ = 119
MIN_BID_ADJ = 6
dfs = []
dfs1 = calcGenderAgeDf(merged_df)
for min_bid_adjsutment in xrange(MIN_BID_ADJ,10):
    for max_bid_adjsutment in xrange(11,MAX_BID_ADJ+1):
        print min_bid_adjsutment/10.0,max_bid_adjsutment/10.0
        dfs2 = calcBidAdjustments(dfs1,min_bid_adjsutment/10.0,max_bid_adjsutment/10.0)
        dfs.append(pd.concat(dfs2))

0.6 1.1
0.6 1.2
0.6 1.3
0.6 1.4
0.6 1.5
0.6 1.6
0.6 1.7
0.6 1.8
0.6 1.9
0.6 2.0
0.6 2.1
0.6 2.2
0.6 2.3
0.6 2.4
0.6 2.5
0.6 2.6
0.6 2.7
0.6 2.8
0.6 2.9
0.6 3.0
0.6 3.1
0.6 3.2
0.6 3.3
0.6 3.4
0.6 3.5
0.6 3.6
0.6 3.7
0.6 3.8
0.6 3.9
0.6 4.0
0.6 4.1
0.6 4.2
0.6 4.3
0.6 4.4
0.6 4.5
0.6 4.6
0.6 4.7
0.6 4.8
0.6 4.9
0.6 5.0
0.6 5.1
0.6 5.2
0.6 5.3
0.6 5.4
0.6 5.5
0.6 5.6
0.6 5.7
0.6 5.8
0.6 5.9
0.6 6.0
0.6 6.1
0.6 6.2
0.6 6.3
0.6 6.4
0.6 6.5
0.6 6.6
0.6 6.7
0.6 6.8
0.6 6.9
0.6 7.0
0.6 7.1
0.6 7.2
0.6 7.3
0.6 7.4
0.6 7.5
0.6 7.6
0.6 7.7
0.6 7.8
0.6 7.9
0.6 8.0
0.6 8.1
0.6 8.2
0.6 8.3
0.6 8.4
0.6 8.5
0.6 8.6
0.6 8.7
0.6 8.8
0.6 8.9
0.6 9.0
0.6 9.1
0.6 9.2
0.6 9.3
0.6 9.4
0.6 9.5
0.6 9.6
0.6 9.7
0.6 9.8
0.6 9.9
0.6 10.0
0.6 10.1
0.6 10.2
0.6 10.3
0.6 10.4
0.6 10.5
0.6 10.6
0.6 10.7
0.6 10.8
0.6 10.9
0.6 11.0
0.6 11.1
0.6 11.2
0.6 11.3
0.6 11.4
0.6 11.5
0.6 11.6
0.6 11.7
0.6 11.8
0.6 11.9
0.7 1.1
0.7 1.2
0.7 1.3
0.7 1.4
0.7 1.5
0.7 1.6
0.7 1.7
0.7 1.8
0.7 1.9
0.7 2.0
0.7 2.1
0.7 2.2
0.7 2.3
0.7 

In [122]:
final_df = pd.concat(dfs)

In [123]:
final_df.to_excel('D:\\_Docs_\\2016-07\\2016-07-26 Gender Age Adjustments\\stranao3_2.xlsx')

In [207]:
calc_mobile_df = merged_df[['campaignId','device_type','sum','visits','goal_visits']].replace(to_replace={'device_type':{'tablet':'mobile'}}).groupby(['campaignId','device_type'],as_index=False).sum()
calc_mobile_df['conversed_cpa'] = calc_mobile_df['goal_visits']/calc_mobile_df['sum']
calc_mobile_df['conversed_cpa'] = calc_mobile_df['conversed_cpa'].replace(np.inf, np.nan).fillna(0)
calc_mobile_df['conversion_rate'] = calc_mobile_df['goal_visits']/calc_mobile_df['visits']
calc_mobile_df['cpv'] = calc_mobile_df['sum']/calc_mobile_df['visits']
calc_mobile_visits = calc_mobile_df['visits'].sum()
calc_mobile_sum = calc_mobile_df['sum'].sum()
calc_mobile_goal_visits = calc_mobile_df['goal_visits'].sum()
calc_mobile_conversion_rate = calc_mobile_goal_visits/calc_mobile_visits
calc_mobile_conversed_cpa = calc_mobile_goal_visits/calc_mobile_sum
calc_mobile_conversed_cpa_std = calc_mobile_df['conversed_cpa'].std()
calc_mobile_cpa = calc_mobile_sum/calc_mobile_goal_visits
print 'Source metrics:'
print 'Visits:',calc_mobile_visits
print 'Sum:',calc_mobile_sum
print 'Goal Visits:',calc_mobile_visits
print 'Conversion Rate',calc_mobile_conversion_rate*100,'%'
print 'Conversed CPA:',calc_mobile_conversed_cpa*100,'%'
print 'Conversed CPA Standart Deviation:',calc_mobile_conversed_cpa_std*100,'%'
print 'CPA:',calc_mobile_cpa
print ''
calc_mobile_df['sqrt_sum'] = calc_mobile_df['sum'].apply(numpy.sqrt)
calc_mobile_df['z_criteria'] = (calc_mobile_df['conversed_cpa'] - calc_mobile_conversed_cpa)/(calc_mobile_conversed_cpa_std/calc_mobile_df['sqrt_sum'])
calc_mobile_df['z_criteria'] = calc_mobile_df['z_criteria'].replace(np.inf, np.nan).fillna(0)
m_maximum_z_criteria = calc_mobile_df['z_criteria'].max()
m_minimum_z_criteria = calc_mobile_df['z_criteria'].min()
m_max_bid_adjustment = 1.44
m_min_bid_adjustment = 0.67
m_delta_z_criteria = m_maximum_z_criteria - m_minimum_z_criteria
m_adjustment_step = (m_max_bid_adjustment-m_min_bid_adjustment)/m_delta_z_criteria
print 'New metrics:'
calc_mobile_df['bid_adjustment'] = (calc_mobile_df['z_criteria'] + abs(m_minimum_z_criteria))*m_adjustment_step + m_min_bid_adjustment
for i in xrange(0,len(calc_mobile_df)):
    if (calc_mobile_df.loc[i,'device_type'] == 'desktop'):
        calc_mobile_df.loc[i,'bid_adjustment'] = 1
calc_mobile_df['new_visits'] = calc_mobile_df['visits']*calc_mobile_df['bid_adjustment']
calc_mobile_df['new_cpv'] = calc_mobile_df['cpv']*calc_mobile_df['bid_adjustment']
calc_mobile_df['new_sum'] = calc_mobile_df['new_visits']*calc_mobile_df['new_cpv']
calc_mobile_df['new_goal_visits'] = calc_mobile_df['new_visits']*calc_mobile_df['conversion_rate']
calc_mobile_df['new_conversed_cpa'] = calc_mobile_df['new_goal_visits']/calc_mobile_df['new_sum']
calc_mobile_df['new_conversed_cpa'] = calc_mobile_df['new_conversed_cpa'].replace(np.inf, np.nan).fillna(0)
calc_mobile_new_visits = calc_mobile_df['new_visits'].sum()
calc_mobile_new_sum = calc_mobile_df['new_sum'].sum()
calc_mobile_new_goal_visits = calc_mobile_df['new_goal_visits'].sum()
calc_mobile_new_conversion_rate = calc_mobile_new_goal_visits/calc_mobile_new_visits
calc_mobile_new_conversed_cpa = calc_mobile_new_goal_visits/calc_mobile_new_sum
calc_mobile_new_conversed_cpa_std = calc_mobile_df['new_conversed_cpa'].std()
calc_mobile_new_cpa = calc_mobile_new_sum/calc_mobile_new_goal_visits
print 'Visits:',calc_mobile_new_visits
print 'Sum:',calc_mobile_new_sum
print 'Goal Visits:',calc_mobile_new_goal_visits
print 'Conversion Rate',calc_mobile_new_conversion_rate*100,'%'
print 'Conversed CPA:',calc_mobile_new_conversed_cpa*100,'%'
print 'Conversed CPA Standart Deviation:',calc_mobile_new_conversed_cpa_std*100,'%'
print 'CPA:',calc_mobile_new_cpa

Source metrics:
Visits: 3427.0
Sum: 56360.4
Goal Visits: 3427.0
Conversion Rate 4.2019258827 %
Conversed CPA: 0.255498541529 %
Conversed CPA Standart Deviation: 0.283134676061 %
CPA: 391.391666667

New metrics:
Visits: 3434.94909511
Sum: 57431.5481375
Goal Visits: 148.441928344
Conversion Rate 4.32151756062 %
Conversed CPA: 0.258467572541 %
Conversed CPA Standart Deviation: 0.271044471958 %
CPA: 386.895729382


,campaignId,device_type,sum,visits,goal_visits
0,15808246,desktop,10243.5,650,16
1,15808246,mobile,5738.4,358,16
2,15808505,desktop,488.4,49,3
3,15808505,mobile,83.1,15,0
4,15808840,desktop,11826.0,510,75
5,15808840,mobile,5745.6,277,26
6,15883169,desktop,17016.0,1201,4
7,15883169,mobile,3885.0,276,1
8,16510865,desktop,886.8,57,0
9,16510865,mobile,447.6,34,3
